In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ef429c1120640dd99f5a28402ac5a220a04611ae10dd643c9e3f5c6a6eea04f8
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [3]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_withcolumn')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

In [4]:
spark

In [5]:
dados=[
    (1, 'Bob', 45, 'M'),
    (2, 'Gloria', 43, 'F'),
    (4, 'Albert', 28, 'M'),
    (5, 'Laura', 33, 'F'),
    (8, 'Simone', 18, 'T'),
    (12, 'Marta', 45, 'F'),
    (45, 'Jairo', 82, 'M'),
    (13, 'Teste', 38, 'T')
]

esquema=["id", "nome", "idade", "gen"]

df = spark.createDataFrame(data=dados, schema=esquema)
df.printSchema()
df.show()

root
 |-- id: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- idade: long (nullable = true)
 |-- gen: string (nullable = true)

+---+------+-----+---+
| id|  nome|idade|gen|
+---+------+-----+---+
|  1|   Bob|   45|  M|
|  2|Gloria|   43|  F|
|  4|Albert|   28|  M|
|  5| Laura|   33|  F|
|  8|Simone|   18|  T|
| 12| Marta|   45|  F|
| 45| Jairo|   82|  M|
| 13| Teste|   38|  T|
+---+------+-----+---+



In [6]:
#Limpando a tabela
df1 = df
df1 = df1.filter(F.col("id") != 13) #Removendo linha desnecessária
df1.show()

+---+------+-----+---+
| id|  nome|idade|gen|
+---+------+-----+---+
|  1|   Bob|   45|  M|
|  2|Gloria|   43|  F|
|  4|Albert|   28|  M|
|  5| Laura|   33|  F|
|  8|Simone|   18|  T|
| 12| Marta|   45|  F|
| 45| Jairo|   82|  M|
+---+------+-----+---+



In [7]:
#Verificar pessoas que possuem mais de 30 anos
df1.filter(F.col("idade") > 30).show()

+---+------+-----+---+
| id|  nome|idade|gen|
+---+------+-----+---+
|  1|   Bob|   45|  M|
|  2|Gloria|   43|  F|
|  5| Laura|   33|  F|
| 12| Marta|   45|  F|
| 45| Jairo|   82|  M|
+---+------+-----+---+



In [8]:
#Verificar apenas as pessoas do sexo F
df1.filter(F.col("gen") == "F").show()

+---+------+-----+---+
| id|  nome|idade|gen|
+---+------+-----+---+
|  2|Gloria|   43|  F|
|  5| Laura|   33|  F|
| 12| Marta|   45|  F|
+---+------+-----+---+



In [9]:
#Verifique a quantidade de pessoas por sexo
df1.groupBy(F.col("gen")).count().show()

+---+-----+
|gen|count|
+---+-----+
|  F|    3|
|  T|    1|
|  M|    3|
+---+-----+



In [10]:
#Calcular a média de idade das pessoas
df1.select(F.avg("idade").alias("mediaDasIdades")).show()

+--------------+
|mediaDasIdades|
+--------------+
|          42.0|
+--------------+



In [11]:
#Fazer uma seleção apenas do nome e idade das pessoas ordenando por nome

(df1.select(F.col("nome"), F.col("idade"))
  .orderBy(F.col("nome"))
  .show())

+------+-----+
|  nome|idade|
+------+-----+
|Albert|   28|
|   Bob|   45|
|Gloria|   43|
| Jairo|   82|
| Laura|   33|
| Marta|   45|
|Simone|   18|
+------+-----+



In [13]:
#Mostrar um ranking de pessoas por idade (Do mais novo para o mais velho)

#Jeito1
(df1.select(F.col("nome"))
  .orderBy(F.col("idade"))
  .show())

#Jeito2
w0 = Window.orderBy("idade")
df1.withColumn("rank", F.row_number().over(w0)).show()

+------+
|  nome|
+------+
|Simone|
|Albert|
| Laura|
|Gloria|
|   Bob|
| Marta|
| Jairo|
+------+

+---+------+-----+---+----+
| id|  nome|idade|gen|rank|
+---+------+-----+---+----+
|  8|Simone|   18|  T|   1|
|  4|Albert|   28|  M|   2|
|  5| Laura|   33|  F|   3|
|  2|Gloria|   43|  F|   4|
|  1|   Bob|   45|  M|   5|
| 12| Marta|   45|  F|   6|
| 45| Jairo|   82|  M|   7|
+---+------+-----+---+----+

